In [1]:
import numpy as np
import pandas as pd

names = ['user_id', 'item_id', 'rating', 'timestamp']
df =pd.read_csv('u.data', sep='\t', names=names)
df.head()


n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print (str(n_users) + ' users')
print (str(n_items) + ' items')

943 users
1682 items


In [2]:
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
ratings

sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print ('Sparsity: {:4.2f}%'.format(sparsity))

Sparsity: 6.30%


In [4]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[0, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

train, test = train_test_split(ratings)

In [6]:
def fast_similarity(ratings, kind='user'):
    if kind == 'user':
        sim = ratings.dot(ratings.T)
    elif kind == 'item':
        sim = ratings.T.dot(ratings)
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return sim / norms / norms.T

In [7]:
#%timeit slow_user_similarity(train)
fast_similarity(train, kind='user')

array([[ 1.        ,  0.16711628,  0.03444883, ...,  0.15130324,
         0.17658202,  0.39360764],
       [ 0.16711628,  1.        ,  0.11264639, ...,  0.16183775,
         0.17358328,  0.10646181],
       [ 0.03444883,  0.11264639,  1.        , ...,  0.05612672,
         0.13633643,  0.0116145 ],
       ..., 
       [ 0.15130324,  0.16183775,  0.05612672, ...,  1.        ,
         0.07530127,  0.09550774],
       [ 0.17658202,  0.17358328,  0.13633643, ...,  0.07530127,
         1.        ,  0.16639724],
       [ 0.39360764,  0.10646181,  0.0116145 , ...,  0.09550774,
         0.16639724,  1.        ]])

In [9]:
user_similarity = fast_similarity(train, kind='user')
item_similarity = fast_similarity(train, kind='item')
print (item_similarity[:4, :4])

[[ 1.          0.35632609  0.28927645  0.42862895]
 [ 0.35632609  1.          0.27373228  0.48775209]
 [ 0.28927645  0.27373228  1.          0.3137646 ]
 [ 0.42862895  0.48775209  0.3137646   1.        ]]


In [10]:
def predict_fast_simple(ratings, similarity, kind='user'):
    if kind == 'user':
        return similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif kind == 'item':
        return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [12]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

item_prediction = predict_fast_simple(train, item_similarity, kind='item')
user_prediction = predict_fast_simple(train, user_similarity, kind='user')

print ('User-based CF MSE: ' + str(get_mse(user_prediction, test)))
print ('Item-based CF MSE: ' + str(get_mse(item_prediction, test)))

User-based CF MSE: 7.17976744284
Item-based CF MSE: 9.88265640226


In [15]:
def predict_topk(ratings, similarity, kind='user', k=40):
    pred = np.zeros(ratings.shape)
    if kind == 'user':
        for i in range(ratings.shape[0]):
            top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
            for j in range(ratings.shape[1]):
                pred[i, j] = similarity[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
                pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))
    if kind == 'item':
        for j in range(ratings.shape[1]):
            top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in range(ratings.shape[0]):
                pred[i, j] = similarity[j, :][top_k_items].dot(ratings[i, :][top_k_items].T) 
                pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items]))        
    
    return pred

In [16]:
pred = predict_topk(train, user_similarity, kind='user', k=40)
print ('Top-k User-based CF MSE: ' + str(get_mse(pred, test)))

Top-k User-based CF MSE: 3.95088018388


In [17]:
pred

array([[ 3.68155863,  1.94142385,  1.56382315, ...,  0.        ,
         0.0731557 ,  0.08054584],
       [ 2.15818073,  0.        ,  0.39255581, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.05784684,
         0.        ,  0.        ],
       ..., 
       [ 3.9032819 ,  0.        ,  0.55166725, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.24297809,  0.65743059,  0.09315032, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.80318421,  2.26269983,  1.42014806, ...,  0.        ,
         0.07231748,  0.        ]])

In [18]:
pred.shape

(943, 1682)

In [20]:
idx_to_movie = {}
with open('u.item', 'r') as f:
    for line in f.readlines():
        info = line.split('|')
        idx_to_movie[int(info[0])-1] = info[1]
idx_to_movie

{0: 'Toy Story (1995)',
 1: 'GoldenEye (1995)',
 2: 'Four Rooms (1995)',
 3: 'Get Shorty (1995)',
 4: 'Copycat (1995)',
 5: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 6: 'Twelve Monkeys (1995)',
 7: 'Babe (1995)',
 8: 'Dead Man Walking (1995)',
 9: 'Richard III (1995)',
 10: 'Seven (Se7en) (1995)',
 11: 'Usual Suspects, The (1995)',
 12: 'Mighty Aphrodite (1995)',
 13: 'Postino, Il (1994)',
 14: "Mr. Holland's Opus (1995)",
 15: 'French Twist (Gazon maudit) (1995)',
 16: 'From Dusk Till Dawn (1996)',
 17: 'White Balloon, The (1995)',
 18: "Antonia's Line (1995)",
 19: 'Angels and Insects (1995)',
 20: 'Muppet Treasure Island (1996)',
 21: 'Braveheart (1995)',
 22: 'Taxi Driver (1976)',
 23: 'Rumble in the Bronx (1995)',
 24: 'Birdcage, The (1996)',
 25: 'Brothers McMullen, The (1995)',
 26: 'Bad Boys (1995)',
 27: 'Apollo 13 (1995)',
 28: 'Batman Forever (1995)',
 29: 'Belle de jour (1967)',
 30: 'Crimson Tide (1995)',
 31: 'Crumb (1994)',
 32: 'Desperado (1995)',
 33: 'D

In [21]:
def top_k_movies(similarity, mapper, movie_idx, k=6):
    return [mapper[x] for x in np.argsort(similarity[movie_idx,:])[:-k-1:-1]]

In [24]:
idx = 27 # Toy Story
movies = top_k_movies(item_similarity, idx_to_movie, idx)

In [25]:
movies

['Apollo 13 (1995)',
 'Forrest Gump (1994)',
 'E.T. the Extra-Terrestrial (1982)',
 'Raiders of the Lost Ark (1981)',
 'Back to the Future (1985)',
 'Jurassic Park (1993)']